### This notebook contains skeleton code, and discusses some ideas on how to approach the problem of designing a robot control architecuture using BLE.

# A Robot Control Class
Classes provide a means of bundling data and functionality together. It helps organize your thoughts and your code. 

Below is the skeleton of one possible class structure to deal with robot communication. You do not have to use the exact same structure, it merely serves as a reference.

In [ ]:
class RobotControl():
    # Initialize Function
    def __init__(self, ble):
        self.ble = ble
        
        # A variable to store the latest sensor value
        self.latest_tof_front_reading = None
        
        # A list to store the history of all the sensor values
        # Each item in the list is a tuple (value, time)
        # WARNING: The list could grow really fast; you need to deal with this accordingly.
        self.tof_readings = []
        
        # A variable to store the latest imu reading
        self.latest_imu_reading = None
        
        # Activate notifications (if required)
        self.setup_notify()
    
    # A function to activate various notifications (if required)
    def setup_notify(self):
        # Code to setup various notify events
        # Ex:
        # ble.start_notify(ble.uuid['RX_TOF2'], self.tof_callback_handler)
        # ble.start_notify(ble.uuid['RX_FANCY_SENSOR'], self.fancy_sensor_callback_handler)
    
    # An example function callback handler for storing the history of the tof sensor
    # Your callback handlers should perform minimal processing!
    # Do not add a receive_* function inside the callback handler, it defeats the purpose of BLE notify
    def tof_callback_handler(self, uuid, byte_array):
        # Append a tuple (value, time) to a list
        self.tof_readings.append( ( self.ble.bytearray_to_float(byte_array), time.time() ) )
    
    # An example function to fetch the front TOF sensor reading
    # Here we assume RX_TOF1 is a valid UUID defined in connection.yaml and
    # in the Arduino code as well
    def get_front_tof(self):
        self.latest_tof_front_reading = self.ble.receive_float(self.ble.uuid['RX_TOF1'])
        pass
    
    # An example function to fetch the IMU readings as a string
    # Here we assume RX_IMU is a valid UUID defined in connection.yaml and
    # in the Arduino code as well
    def get_imu(self):
        self.latest_imu_reading = self.ble.receive_string(self.ble.uuid['RX_IMU'])
        pass
    
    # A function to instruct the robot to stop 300mm from wall
    def wall_pid(self):
        ble.send_command(CMD.WALL_PID)
    
    
    # A function to instruct the robot to move forward
    def move_forward(self, speed):
        # Code to move forward
        ble.send_command(CMD.MOVE_FORWARD, speed)
    
    # A function to stop robot motion
    def stop(self):
        # Code to stop robot motion
        ble.send_command(CMD.STOP)


### You can now use such a class to control your robot

In [ ]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

# Instantiate RobotControl class
rc = RobotControl(ble)

### Move robot forward for 3 secs and get sensor readings ###
rc.get_imu()

rc.move_forward(50)
log.info("IMU Reading: " + str(rc.latest_imu_reading))
await time.sleep(3)
rc.stop()

rc.get_imu()
log.info("IMU Reading: " + str(rc.latest_imu_reading))

## Setup:

In [10]:
%load_ext autoreload
%autoreload 2

from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import time
import numpy as np
import asyncio


LOG.propagate = False

ARR_SIZE = 200
feature_names_list = ["TOF Readings", "Time Stamps", "Motor Percentages"]
data_list = [ [] for _ in range(len(feature_names_list))]

feature_ind = 0

def data_receiver(uuid, byte_arr):
    fl = ble.bytearray_to_float(byte_arr)
    if len(data_list[feature_ind]) < ARR_SIZE:
        data_list[feature_ind].append(fl)
    elif feature_ind < len(feature_names_list)-1:
        feature_ind = feature_ind + 1
        data_list[feature_ind].append(fl)
    else: raise("More data than expected.")
        

    

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ModuleNotFoundError: No module named 'bleak'

# PID execution:

In [9]:

# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

# Instantiate RobotControl class
rc = RobotControl(ble)

rc.wall_pid()
time.sleep(5)
ble.start_notify(ble.UUID['RX_FLOAT'], data_receiver)


NameError: name 'get_ble_controller' is not defined

<hr>

### There are two possible approaches to reading values from the Artemis board
**Approach 1:** Read values explicitly <br>
**Approach 2:** Notifications

#### You can use a combination of both i.e read some values explicitly and activate notifications for others.

## Approach 1: Read values explicitly
Below is a possible structure to run your robot commands in a loop and explicitly read the sensor values as required. 

You have more control of your code in this approach. However, the read functions could perform slower in comparison to notify events.

In [ ]:
# Add this to the top most cell containing the imports
import asyncio


while True:
    ###### Your code ######
    # Ex: Move the robot for 1 sec
    # rc.move_forward(50)
    # await asyncio.sleep(1)
    # rc.stop()
    
    ###### Read values ######
    # rc.get_imu()
    # rc.get_front_tof()

## Approach 2: Notifications
Below is a possible structure to run your robot command in a loop and utilize the notify events. 

You may have to tweak the sleep time based on how you program your arduino, system OS, system specs, system load and, the number and frequency of notify events. 

However, you **don't have to worry too much** about this if you keep your notify callback functions light. It becomes an issue only when you have a large number of notify events within a (very) short duration of time.

In [ ]:
while True:
    ###### Your code ######
    # Ex: Move the robot for 1 sec
    # rc.move_forward(50)
    # await asyncio.sleep(1)
    # rc.stop()
    
    # Process notify events
    # Sleep the current execution so that the notify callback functions can run
    # The sleep time can be much smaller, but it depends on the OS, hardware specs, number of notify events and the system load
    await asyncio.sleep(0.1)